In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from random import shuffle
import zipfile


pd.set_option('display.max_colwidth', -1)

In [3]:
df=pd.read_csv('processed_train_data.csv',engine='python')

In [4]:
df.head()

,Webpage_id,Domain,Url,Tag,Html
0,1,www.fiercepharma.com,http://www.fiercepharma.com/marketing/tecfidera-gilenya-and-aubagio-s-3-way-battle-for-ms-share-about-to-get-more-interesting,news,"Tecfidera, Gilenya and Aubagio&#039;s 3-way battle for MS share is about to heat up | FiercePharma"
1,2,www.fiercepharma.com,http://www.fiercepharma.com/pharma/novo-equipped-to-weather-storm-u-s-diabetes-market-ceo-says,news,"Novo equipped to weather the storm in the U.S. diabetes market, CEO says | FiercePharma"
2,3,www.fiercepharma.com,http://www.fiercepharma.com/pharma/another-exec-departs-troubled-endo-and-time-it-s-for-another-drugmaker,news,"Another exec departs troubled Endo--and this time, it&#039;s for another drugmaker | FiercePharma"
3,4,www.fiercepharma.com,http://www.fiercepharma.com/pharma/teva-buy-biosim-specialist-celltrion-it-wouldn-t-say-no,news,Would Teva buy Korea&#039;s Celltrion to beef up in biosimilars? It wouldn&#039;t say no | FiercePharma
4,5,www.fiercepharma.com,http://www.fiercepharma.com/marketing/actress-marissa-tomei-partners-allergan-restasis-to-drive-dry-eye-awareness,news,Restasis-maker Allergan recruits actress Marisa Tomei to drive dry eye awareness | FiercePharma


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53447 entries, 0 to 53558
Data columns (total 5 columns):
Webpage_id    53447 non-null object
Domain        53447 non-null object
Url           53447 non-null object
Tag           53447 non-null object
Html          53447 non-null object
dtypes: object(5)
memory usage: 2.4+ MB


In [10]:
target=df.Tag.values
df.dropna(axis=0,subset=['Tag'],inplace=True)
df.Html.fillna(value='missing',inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53447 entries, 0 to 53558
Data columns (total 5 columns):
Webpage_id    53447 non-null object
Domain        53447 non-null object
Url           53447 non-null object
Tag           53447 non-null object
Html          53447 non-null object
dtypes: object(5)
memory usage: 2.4+ MB


In [12]:
def preprocess(url,title):
    
    pattern=r'//.*'
    urls=[]
    for i in url:
        b=re.findall(pattern,i)
        c=b[0][2:]
        d=re.split('\.|/|-|_',c)
        e=''
        for i in d:
            e+=str(i).lower()
            e+=' '
        e = ''.join([i for i in e if not i.isdigit()])
        f=''
        for i in e:
            if i.isalpha() or i==' ':
                f+=i
            else:
                f+=' '
        urls.append(f)
        
    titles=[]
    for e in title:    
        e = ''.join([i for i in e if not i.isdigit()])
        f=''
        for i in e:
            if i.isalpha() or i==' ':
                #print(i)
                f+=i
            else:
                f+=' '
        f = re.sub(' +',' ',f) # replace series of spaces with single space
        titles.append(f)
    
    data=[]
    for i in range(len(title)):
        s=titles[i]+' '+urls[i]
        data.append(str(s))
        
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(urls)
    
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    
    return(data)

In [13]:
%%time
urls=preprocess(df.Url,df.Html)
#target=df.Tag.values

Wall time: 6.28 s


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53447 entries, 0 to 53558
Data columns (total 5 columns):
Webpage_id    53447 non-null object
Domain        53447 non-null object
Url           53447 non-null object
Tag           53447 non-null object
Html          53447 non-null object
dtypes: object(5)
memory usage: 2.4+ MB


In [16]:
urls[:1]

['Tecfidera Gilenya and Aubagio s way battle for MS share is about to heat up FiercePharma www fiercepharma com marketing tecfidera gilenya and aubagio s  way battle for ms share about to get more interesting ']

In [17]:
target[:5]

array(['news', 'news', 'news', 'news', 'news'], dtype=object)

In [18]:
from sklearn.linear_model import SGDClassifier

text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=0.001, n_iter=5, random_state=42))])

text_clf_svm = text_clf_svm.fit(urls, target)
predicted_svm = text_clf_svm.predict(urls)
np.mean(predicted_svm == target)

C:\Users\Nishant\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.8843527232585552

In [19]:
from sklearn.linear_model import SGDClassifier
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(urls)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    
svm=SGDClassifier(loss='hinge', penalty='l2',alpha=0.001, n_iter=5, random_state=42)

svm.fit(X_train_counts,target)
predicted= svm.predict(X_train_counts)
np.mean(predicted==target)

C:\Users\Nishant\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.9585757853574569

In [25]:
X_train_tfidf.shape[1]

46298

In [22]:
target.shape

(53447,)

# Pytorch

In [115]:
import torch 
from torch.autograd import Variable
import torch.nn as nn

In [116]:
def get_batch(X_train_tfidf,target,i,batch_size):
    batch_data = X_train_tfidf[i*batch_size:i*batch_size+batch_size]
    batch_target = target[i*batch_size:i*batch_size+batch_size]
    
    return(np.array(batch_data),np.array(batch_target))

    
    

In [117]:
learning_rate=0.1
epochs=5
batch_size=100

input_size=X_train_tfidf.shape[1]
hidden_size=100
output_size=9

In [118]:
class MyNet(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(MyNet,self).__init__()
        self.layer1=nn.Linear(input_size,hidden_size,bias=True)
        self.relu=nn.ReLU()
        #self.layer2=nn.Linear(hidden_size,hidden_size,bias=True)
        self.output_layer=nn.Linear(hidden_size,output_size,bias=True)
        
    def forward(self,x):
        out=self.layer1(x)
        out=self.relu(out)
        out=self.output_layer(out)
        return(out)
    

In [119]:
net=MyNet(input_size,hidden_size,output_size)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(net.parameters(),lr=learning_rate)


In [56]:
from scipy.sparse import coo_matrix

for epoch in range(epochs):
    total_batch=int(X_train_tfidf.shape[0]/batch_size)
    
    for i in range(total_batch):
        batch_data,batch_target=get_batch(X_train_tfidf,target,i,batch_size)
        text=Variable(torch.sparse.FloatTensor(batch_data))
        #torch.sparse.FloatTensor(indices, values, size)
        tag=Variable(torch.sparse.FloatTensor(batch_target))
        
        optimizer.zero_grad()
        output=net.forward(text)
        loss=criterion(output,tag)
        loss.backward()
        optimizer.step()
        
        print(loss.data[0])
        

TypeError: new() received an invalid combination of arguments - got (numpy.ndarray), but expected one of:
 * (torch.device device)
      didn't match because some of the arguments have invalid types: ([31;1mnumpy.ndarray[0m)
 * (tuple of ints size, torch.device device)
 * (Tensor indices, Tensor values, torch.device device)
 * (Tensor indices, Tensor values, tuple of ints size, torch.device device)


In [120]:
from scipy.sparse import coo_matrix

batch_data,batch_target=get_batch(X_train_counts,target,0,batch_size)
text=Variable(torch.FloatTensor(batch_data))

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: double, float, float16, int64, int32, and uint8.

In [65]:
a=coo_matrix(batch_data)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all().

In [72]:
X_train_counts

<53447x46298 sparse matrix of type '<class 'numpy.int64'>'
	with 720665 stored elements in Compressed Sparse Row format>

In [35]:
a=X_train_counts[:1000].todense()

In [40]:
a.shape

(1000, 46298)

In [51]:
from scipy.sparse import csr_matrix
S = csr_matrix(X_train_counts)

In [52]:
b=S[:100].todense()

In [88]:
#print(S.data.shape)   #values 
#print(S.indices.shape)       #y coordinate  
print(S.getcol())

(720665,)


In [113]:
print(S[:2].shape)

(2, 46298)


In [48]:
A = np.array([[1, 0, 0, 1, 0, 0], [0, 0, 2, 0, 0, 1], [0, 0, 0, 2, 0, 0]])
print(A)
# convert to sparse matrix (CSR method)
S = csr_matrix(A)
print(S)
# reconstruct dense matrix
B = S.todense()
print(B)

[[1 0 0 1 0 0]
 [0 0 2 0 0 1]
 [0 0 0 2 0 0]]
  (0, 0)	1
  (0, 3)	1
  (1, 2)	2
  (1, 5)	1
  (2, 3)	2
[[1 0 0 1 0 0]
 [0 0 2 0 0 1]
 [0 0 0 2 0 0]]


In [50]:
print(S)

  (0, 0)	1
  (0, 3)	1
  (1, 2)	2
  (1, 5)	1
  (2, 3)	2
